In [1]:
# Standard modules
import time
import logging
import copy
import cPickle as pkl

# Numpy and Scipy
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages

# Pylearn2 and Theano
import theano
import pylearn2
from pylearn2.datasets import preprocessing

# Custom modules
import core_causal_features
import dataset_binary_gratings
import ai_gratings
import helpers

WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


In [2]:
# 1. Set file names.

# creates unique IDs that assign the current date and time to the model, data, report, log
curT = time.localtime()
MODEL_FNAME = 'EXPERIMENTS/GRATINGS_MLP.pkl'.\
              format(curT.tm_year, curT.tm_mon, curT.tm_mday, 
                     curT.tm_hour, curT.tm_min, curT.tm_sec)

DATA_FNAME = 'EXPERIMENTS/Data_GRATING_{0}_{1}_{2}_{3}_{4}_{5}.pkl'.\
              format(curT.tm_year, curT.tm_mon, curT.tm_mday, 
                     curT.tm_hour, curT.tm_min, curT.tm_sec)

REPORT_FNAME = 'EXPERIMENTS/Report_GRATING_{0}_{1}_{2}_{3}_{4}_{5}.pdf'.\
               format(curT.tm_year, curT.tm_mon, curT.tm_mday, 
                      curT.tm_hour, curT.tm_min, curT.tm_sec)

LOGS_FNAME = 'EXPERIMENTS/Logs_GRATING_{0}_{1}_{2}_{3}_{4}_{5}.log'.\
              format(curT.tm_year, curT.tm_mon, curT.tm_mday, 
                     curT.tm_hour, curT.tm_min, curT.tm_sec)

In [3]:
# 2. Configure logging.

#I don't know anything about what the logger does. Ignoring for now 
logging.getLogger('pylearn2').setLevel(logging.ERROR)


In [4]:
# 3. Set other parameters.
IM_SHAPE = (10, 10) #image dimensions 
N_TRAIN = 100 #number of images in training data set 
N_BATCHES = 10 ##TODO 
QUERY_BATCH_SIZE = 100 ##TODO 
RETRAIN = 0 #RETRAIN = 1 means generate XXX for first time. RETRAIN = 0 means use existing
DATA_NOISE = 0.03 #noise level to assign to data 
MANIP_THR = 0.05 ## TODO
MANIP_ALPHA = 0.5 ## TODO 
agent = ai_gratings.Ai_Causal(visual_reshape=IM_SHAPE) #generates AI agent object (not much happens at the initialization step here )


In [5]:
# Seed the rng.
np.random.seed(1423)

# define function to create the data set 
def prepare_datasets(agent, n_training=10000, im_shape=(10,10), noise_lvl=0):
    '''function to create visual bars images. returns tuple containing training set, validation set, and test set'''
    print('Creating an observational gratings dataset...')
    # creates data set object - generates images, assigns the images and the behavior to self 
    data_train = dataset_binary_gratings.GRATINGS(n_samples=n_training*5/4, agent=agent, im_shape=im_shape, noise_lvl=noise_lvl) 

    # Normalize the data.
    # preprocessor = preprocessing.Standardize()
    # data_train.apply_preprocessor(preprocessor, can_fit=True)
    # data_train.X = data_train.X-0.5

    # split the data into training and validation sets 
    data_valid = copy.deepcopy(data_train)
    split_id = int(4*np.floor(data_train.X.shape[0]/5)) 
    data_train.X = data_train.X[:split_id]
    data_train.y = data_train.y[:split_id]
    data_valid.X = data_valid.X[split_id:]
    data_valid.y = data_valid.y[split_id:]
    
    # generate a small test set (100 images) with same parameters as training data 
    data_test = dataset_binary_gratings.GRATINGS(n_samples=100, agent=agent, 
                         im_shape=im_shape, noise_lvl=noise_lvl)
    # data_test.apply_preprocessor(preprocessor, can_fit=False)
    # data_test.X = data_test.X-0.5

    print('Done. Training set size {0}. Validation set size {1}.\n'.format(data_train.y.size,data_valid.y.size))
    return (data_train, data_valid, data_test)


In [6]:
# 4. Create the observational dataset.
(data_train, data_valid, data_test) =prepare_datasets(agent, n_training=N_TRAIN, im_shape=IM_SHAPE, noise_lvl=DATA_NOISE)


Creating an observational gratings dataset...
Done. Training set size 100. Validation set size 25.



In [7]:
# 5. Train an observational MLP or load a pre-trained model.
train_params = {'data_train': data_train,
                'data_valid': data_valid,
                'data_test': data_test,
                'model_fname': MODEL_FNAME,
                'arch': [50, 50],
                'learning_rate': 0.001,
                'max_iters': 250,
                'max_after_best': 100,
                'batch_size': 10}
if RETRAIN:
    # train_mlp = trains a multilayer perceptron w ^parameters
    # (since no model is given, a new MLP is created)
    core_causal_features.train_mlp(**train_params) 

    #then loads the model generated from this process 
    model_obs = pylearn2.utils.serial.load(MODEL_FNAME)
else:

    #otherwise, model is loaded from file 
    model_obs = pylearn2.utils.serial.load('EXPERIMENTS/GRATINGS_MLP.pkl')

print('Observational model trained. Validation error {:.2g}'.\
      format(float(model_obs.monitor.channels['valid_y_misclass'].\
                   val_record[-1])))
data_obs = data_train


Observational model trained. Validation error 0.12


In [20]:
#this is a block jenna added for figuring things out 
print(data_obs.X.shape) #the dimensions of X are (# of images, flattened image dimensions (ie 10x10=100))
print(data_obs.y.shape) # the dimensions of y are (# of images, 1) 
print(data_obs.X_labels) #I'm still trying to figure out where/how the labels on the daya are being stored 
print(data_obs.y_labels)

(100L, 100L)
(100L, 1L)
None
4


In [9]:
# 6. Retrain the model using the Causal Coarsening Theorem
(model_csl, data_csl_train, data_csl_valid) = core_causal_features.causal_coarsening(model_obs=model_obs, data_obs_train=data_obs, data_obs_valid=data_valid, agent=agent, n_queries=10, train_params=train_params)

ValueError: Range cannot be empty (low >= high) unless no samples are taken

In [16]:
print(model_csl, data_csl_train, data_csl_valid)
print(data_csl_train.X.shape)

(<pylearn2.models.mlp.MLP object at 0x000000000B61B408>, <pylearn2.datasets.dense_design_matrix.DenseDesignMatrix object at 0x000000000B60A548>, <pylearn2.datasets.dense_design_matrix.DenseDesignMatrix object at 0x000000000B63C448>)
(100L, 100L)


In [9]:
#Note: this block takes over two hours to execute (gross)

# 8. Learn the manipulator.
for batch_id in range(N_BATCHES):
    print('\n\nCausal query batch {}/{}'.format(batch_id, N_BATCHES))
    # 8.2. Perform the causal queries on current batch, including
    #    manipulated versions on the batch.
    manip_pool = core_causal_features.prepare_img_queries(data_train=data_csl_train,
            model=model_csl, n_queries=QUERY_BATCH_SIZE, im_shape=[1,100],
            alpha=MANIP_ALPHA, augment_thr=MANIP_THR, augment_coeff=np.inf)
    
    # 8.2a. Annotate the queries using a simulated agent and incorporate them into the data.
    (data_train, data_valid) = helpers.incorporate_queries(manip_pool=manip_pool, agent=agent,
            data_train=data_csl_train, data_valid=data_csl_valid) 

    # 8.5. Continue training the classifier, starting from previous
    #     weights, using the current full causal dataset.
    train_params['data_train'] = data_train
    train_params['data_valid'] = data_valid
    train_params['model'] = None 
    core_causal_features.train_mlp(**train_params)
    model_csl = pylearn2.utils.serial.load(train_params['model_fname'])


pulation activation 0.628
Pre-augmentation activation 0.881907665772
Manipulated image 663... Post-manipulation activation 0.95
Pre-augmentation activation 0.896521821796
Manipulated image 531... Post-manipulation activation 0.939
Pre-augmentation activation 0.897289398059
Manipulated image 496... Post-manipulation activation 0.297
Pre-augmentation activation 0.927974150297
Manipulated image 34... Post-manipulation activation 0.92
Pre-augmentation activation 0.925018813464
Manipulated image 276... Post-manipulation activation 0.0431
Pre-augmentation activation 0.909621880015
Manipulated image 261... Post-manipulation activation 0.956
Pre-augmentation activation 0.903155431998
Manipulated image 293... Post-manipulation activation 0.974
Pre-augmentation activation 0.944940003143
Manipulated image 526... Post-manipulation activation 0.949
Pre-augmentation activation 0.939064674473
Manipulated image 597... Post-manipulation activation 0.122
Pre-augmentation activation 0.909007402482
Manipu